# Graphistry Netflow Demo

In this example we are taking millions of rows of netflow (network traffic flow) data in order to search for anomalous activity within a network.

In this notebook, we will: 
- Set up [BlazingSQL](https://blazingsql.com) and the [RAPIDS AI](https://rapids.ai/) suite.
- Query 20M rows of network security data (netflow) with BlazingSQL and then pass to Graphistry to visualize and interact with the data.
![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-39814657-5&cid=555&t=event&ec=guides&ea=graphistry-netflow-demo&dt=graphistry-netflow-demo)

## Setup
### Environment Sanity Check 

RAPIDS packages (BlazingSQL included) require Pascal+ architecture to run. For Colab, this translates to a T4 GPU instance. 

The cell below will let you know what type of GPU you've been allocated, and how to proceed.

In [1]:
!wget https://github.com/BlazingDB/bsql-demos/raw/master/utils/colab_env.py
!python colab_env.py 



***********************************
GPU = b'Tesla T4'
Woo! You got the right kind of GPU!
***********************************




## Installs 
The cell below pulls our Google Colab install script from the `bsql-demos` repo then runs it. The script first installs miniconda, then uses miniconda to install BlazingSQL and RAPIDS AI. This takes a few minutes to run. 

In [3]:
!wget https://github.com/BlazingDB/bsql-demos/raw/master/utils/bsql-colab.sh 
!bash bsql-colab.sh

import sys, os, time
sys.path.append('/usr/local/lib/python3.6/site-packages/')
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'

import subprocess
subprocess.Popen(['blazingsql-orchestrator', '9100', '8889', '127.0.0.1', '8890'],stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
subprocess.Popen(['java', '-jar', '/usr/local/lib/blazingsql-algebra.jar', '-p', '8890'])

import pyblazing.apiv2.context as cont
cont.runRal()
time.sleep(1) 
!wget https://github.com/BlazingDB/bsql-demos/raw/master/utils/colab_env.py
!python colab_env.py 

## Download CSV

The cell below will download the data for this demo from AWS and store it locally as `nf-chunk2.csv`. 

In [ ]:
!wget https://blazingsql-colab.s3.amazonaws.com/netflow_data/nf-chunk2.csv

--2019-08-23 21:43:50--  https://blazingsql-colab.s3.amazonaws.com/netflow_data/nf-chunk2.csv
Resolving blazingsql-colab.s3.amazonaws.com (blazingsql-colab.s3.amazonaws.com)... 52.216.137.76
Connecting to blazingsql-colab.s3.amazonaws.com (blazingsql-colab.s3.amazonaws.com)|52.216.137.76|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2725056295 (2.5G) [text/csv]
Saving to: ‘nf-chunk2.csv’

nf-chunk2.csv       100%[===================>]   2.54G  49.2MB/s    in 56s     

2019-08-23 21:44:46 (46.2 MB/s) - ‘nf-chunk2.csv’ saved [2725056295/2725056295]



## Blazing Context
Here we are importing cuDF and BlazingContext. You can think of the BlazingContext much like a Spark Context (i.e. where information such as FileSystems you have registered and Tables you have created will be stored). If you have issues running this cell, restart runtime and try running it again.

In [1]:
from blazingsql import BlazingContext
import cudf
# start up BlazingSQL
bc = BlazingContext()

BlazingContext ready


### Load & Query Tables

In the cell below, we are first loading the CSV file into a GPU DataFrame (gdf), and then creating tables so that we can run SQL queries on those GDFs. 

Note: when you create a table off of a GDF there is no copy, it is merely registering the schema.

In [2]:
# Load CSVs into GPU DataFrames (gdf)
netflow_gdf = cudf.read_csv('nf-chunk2.csv')

# Create BlazingSQL Tables - There is no copy in this process
bc.create_table('netflow', netflow_gdf)

#### Query
With the table made, we can simply run a SQL query.

We are going to run some joins and aggregations in order to condese these millions of rows into thousands of rows that represent nodes and edges.

In [3]:
%%time
# make a query
query = '''
        SELECT
            a.firstSeenSrcIp as source,
            a.firstSeenDestIp as destination,
            count(a.firstSeenDestPort) as targetPorts,
            SUM(a.firstSeenSrcTotalBytes) as bytesOut,
            SUM(a.firstSeenDestTotalBytes) as bytesIn,
            SUM(a.durationSeconds) as durationSeconds,
            MIN(parsedDate) as firstFlowDate,
            MAX(parsedDate) as lastFlowDate,
            COUNT(*) as attemptCount
        FROM
            netflow a
        GROUP BY
            a.firstSeenSrcIp,
            a.firstSeenDestIp
            '''

# query the table
gdf = bc.sql(query)

CPU times: user 32.3 ms, sys: 453 µs, total: 32.8 ms
Wall time: 438 ms


In [4]:
# how's the dataframe look?
gdf.head(10)

,source,destination,targetPorts,bytesOut,bytesIn,durationSeconds,firstFlowDate,lastFlowDate,attemptCount
0,172.10.1.226,239.255.255.250,3,875,0,6,2013-04-03 06:36:44,2013-04-03 06:36:51,3
1,172.30.1.200,239.255.255.250,9,2275,0,12,2013-04-03 06:35:52,2013-04-03 06:43:26,9
2,172.30.1.225,172.0.0.1,1,90,90,0,2013-04-03 06:36:14,2013-04-03 06:36:14,1
3,172.30.1.46,239.255.255.250,17,4025,0,18,2013-04-03 06:35:22,2013-04-03 06:44:08,17
4,172.20.2.71,239.255.255.250,3,875,0,6,2013-04-03 06:37:11,2013-04-03 06:37:18,3
5,172.10.1.233,172.0.0.1,1,180,180,0,2013-04-03 06:36:45,2013-04-03 06:36:45,1
6,172.30.1.102,10.0.0.10,1,454,633,0,2013-04-03 06:48:05,2013-04-03 06:48:05,1
7,172.20.1.39,239.255.255.250,1,525,0,6,2013-04-03 06:36:59,2013-04-03 06:36:59,1
8,172.10.1.96,172.0.0.1,1,180,0,0,2013-04-03 06:36:21,2013-04-03 06:36:21,1
9,172.20.1.2,239.255.255.250,19,3675,0,6,2013-04-03 06:36:50,2013-04-03 06:36:59,19
